[View in Colaboratory](https://colab.research.google.com/github/djinnome/rogain/blob/master/Co_simul_G.ipynb)

#  End-to-end differentiable architecture to learn the thermodynamic favorability of enzyme-catalyzed reactions 
Motivation:  Successful metabolic engineering of microbial cell factories depends on the thermodynamic favorability of engineered pathways.   Because the number of unique biochemical reactions in a cell is on the order of tens of thousands, any method to calculate the Gibbs free energy of a reaction must be fast.
Because the Gibbs free energy of the reaction is proportional to the log concentrations, the standard free energy of formation for the products and reactants is the dominating source of error. Therefore, any method that calculates the Gibbs free energy of a reaction must also be accurate.

Interestingly, the Gibbs free energy of a reaction can be calculated in two different ways. The solvent approach involves calculating the Gibbs free energy of formation for substrates and products, and then subtracting the former from the latter.  The catalyst approach involves calculating the binding energy of substrate to enzyme, the energy of transition from enzyme-bound substrate to enzyme-bound product, and then the energy of unbinding from enzyme-bound product to free enzyme and product, as shown in the figure below.

![alt text](http://iopscience.iop.org/1478-
3975/14/5/055003/downloadHRFigure/figure/pbaa7d80f02)


 Empirical methods for calculating the Gibbs free energy of metabolites such as the **group contribution method (GCM)** and the **component contribution method (CCM)**  are fast, but not accurate. Quantum chemistry methods such as **Density Functional Theory (DFT)** for calculating Gibbs free energy of formation for each metabolite, and **protein folding** and **molecular docking algorithms** for predicting enzyme-ligand binding energies can be accurate, but not fast. Recently, deep learning methods such as message-passing neural networks for metabolites and 3D convolutional networks for enzyme-ligand binding have  emerged as an important alternative modelling tool for the fast and accurate prediction of Gibbs Free energies.

In this proposal, we apply deep learning to both approaches so that each approach acts as a check on the other.
# Table of Contents



>[Predicting Gibbs free energy of metabolic reactions using Neural Network](#scrollTo=M56jp8HJtLh8)

>[Table of Contents](#scrollTo=M56jp8HJtLh8)

>[Prediction of Gibbs free energy of reaction using the solvent approach](#scrollTo=Y7qRW8gJnXDD)

>>>[Neural net architecture](#scrollTo=Y7qRW8gJnXDD)

>>>[Datasets](#scrollTo=Y7qRW8gJnXDD)

>>>>[Training Data (110K molecules, 13 properties)](#scrollTo=Y7qRW8gJnXDD)

>>>>[Validation and dev data (20K molecules, 13 properties)](#scrollTo=Y7qRW8gJnXDD)

>>>>[Test data (400 molecules, 13 properties)](#scrollTo=Y7qRW8gJnXDD)

>>>>[Molecule Prediction data (3K molecules, 1 property $\Delta G^\circ_{f}$)](#scrollTo=Y7qRW8gJnXDD)

>>>>[Reaction Prediction data (1200 reactions, 1 property $\Delta G^\circ_{rxn}$)](#scrollTo=Y7qRW8gJnXDD)

>>>>[Experimental (Golden set) data (~100 reactions)](#scrollTo=Y7qRW8gJnXDD)

>>>[Accuracy](#scrollTo=Y7qRW8gJnXDD)

>>>[Source code](#scrollTo=Y7qRW8gJnXDD)

>>>[Proposed work](#scrollTo=Y7qRW8gJnXDD)

>[Predicting Gibbs free energy of the reaction using the catalyst approach](#scrollTo=9JKl2mO_oP4C)

>[Workflow from protein sequence and molecular structure to enzyme-substrate and enzyme-product binding affinities](#scrollTo=9JKl2mO_oP4C)

>>[Direct prediction of binding affinity from protein sequence and chemical structure using PADME](#scrollTo=9JKl2mO_oP4C)

>>>[Neural network architecture](#scrollTo=9JKl2mO_oP4C)

>>>[Datasets](#scrollTo=9JKl2mO_oP4C)

>>>[Accuracy](#scrollTo=9JKl2mO_oP4C)

>>>[Source code](#scrollTo=9JKl2mO_oP4C)

>>>[Proposed work](#scrollTo=9JKl2mO_oP4C)

>>[Protein sequence to 3D structure using Recurrent Geometric Network](#scrollTo=9JKl2mO_oP4C)

>>>[Neural network architecture](#scrollTo=9JKl2mO_oP4C)

>>>[Datasets](#scrollTo=9JKl2mO_oP4C)

>>>[Accuracy](#scrollTo=9JKl2mO_oP4C)

>>>[Source Code](#scrollTo=9JKl2mO_oP4C)

>>>[Proposed work](#scrollTo=9JKl2mO_oP4C)

>>[Ligand binding pocket of 3D structure using DeepSite](#scrollTo=9JKl2mO_oP4C)

>>>[Neural network architecture](#scrollTo=9JKl2mO_oP4C)

>>>[Datasets](#scrollTo=9JKl2mO_oP4C)

>>>[Accuracy](#scrollTo=9JKl2mO_oP4C)

>>>[Source code](#scrollTo=9JKl2mO_oP4C)

>>>[Proposed work](#scrollTo=9JKl2mO_oP4C)

>>[Ligand Pose inside 3D pocket using Gnina](#scrollTo=L9hBFV_EzABU)

>>>[Neural network architecture](#scrollTo=L9hBFV_EzABU)

>>>[Dataset](#scrollTo=L9hBFV_EzABU)

>>>[Accuracy](#scrollTo=L9hBFV_EzABU)

>>[Binding affinity of Protein-Ligand complex using Atomic Convolutional Network](#scrollTo=L9hBFV_EzABU)

>>>[Neural net architecture](#scrollTo=L9hBFV_EzABU)

>>>[Dataset](#scrollTo=L9hBFV_EzABU)

>>>[Accuracy](#scrollTo=L9hBFV_EzABU)

>>>[Source code](#scrollTo=L9hBFV_EzABU)

>>>[Proposed work](#scrollTo=L9hBFV_EzABU)

>>[Metrics:](#scrollTo=CNqqZY16nXDF)




# Prediction of Gibbs free energy of reaction using the solvent approach

To calculate the Gibbs free energy of a reaction using the solvent approach, 

$$\Delta G_{rxn} =S^T\Delta G^\circ_f + RT\cdot S^T\log c$$
where $S$ is the stoichiometric matrix of reactions and metabolites, $\Delta G^\circ_f$ is the standard free energy of formation of the reactants and products, $R$ is the gas constant, $T$ is the temperature and $\log c$ is the log concentrations of the reactants and products.

Previous work has shown that the ability to calculate the Gibbs free energy of formation for each metabolite using Message Passing Neural Networks (MPNN) is within chemical accuracy of a DFT. 

![DFT vs MPNN](https://github.com/djinnome/rogain/raw/master/images/MPNN-vs-DFT.png)

### Neural net architecture
Supervised learning on molecules has seen rapid improvements with applications to chemistry, drug discovery, and materials science
* Message-Passing Neural Networks ([Gilmer et al 2017](https://arxiv.org/pdf/1704.01212.pdf)) A framework for describing many graph neural networks (described below) in terms of  Message, Update and  Readout operations on graphs with analogy to message passing in Probabilistic Graphical models
  * Graph Convolutional Networks  ([Thomas Kipf et al 2016]
(http://arxiv.org/abs/1609.02907))
  * Gated Graph Convolutional Networks [(Li et al 2016)](http://arxiv.org/abs/1511.05493)
  * [Interaction Networks](https://github.com/PNNL-CompBio/graph-neural-networks) ([Battaglia et al 2016](http://arxiv.org/abs/1612.00222)) This is the network we got working first
  * [Deep Tensor Neural Networks](https://github.com/atomistic-machine-learning/dtnn) [(Schutt 2017a)](https://www.nature.com/articles/ncomms13890) This was referenced in MPNN paper
  * [SchNet](https://github.com/djinnome/SchNet) ([Schutt 2017b](http://arxiv.org/abs/1712.06113)) This improved on DTNN
  * Neural Message Passing with Edge Updates [(Bjorgensen 2018)](https://arxiv.org/pdf/1806.03146.pdf) This improved on SchNet.
  * Graph Networks[(Battaglia et al 2018)](https://arxiv.org/abs/1806.01261) This is a generalization of MPNNs (see figure below)
* Ensemble networks to predict properties. According to the MPNN paper: 
      We were able to further improve performance on the test set by ensembling the predictions of the five models with lowest validation error.
![graph neural network architectures  Figure 4 from (Battagila et al 2018)](https://ndownloader.figshare.com/files/12245093/preview/12245093/preview.jpg?private_link=7bc3719fab09b0639bd4)

### Datasets

#### Training Data (110K molecules, 13 properties)
* Graph: Quantum mechanics data (QM9/B3LYP) raw distances and chemical graph 
* Labels: 13 properties

    
#### Validation and dev data (20K molecules, 13 properties)
* Graph: Quantum mechanics data (QM9/B3LYP) raw distances and chemical graph
* Labels: 13 properties

#### Test data (400 molecules, 13 properties)
* MetaCyc subset that is in QM9
* Equilibrator subset that is in QM9

#### Molecule Prediction data (3K molecules, 1 property $\Delta G^\circ_{f}$)
* MetaCyc (Group Contribution) subset that is not in the QM9 data and has <= 9 carbons and only consists of (C H N O F) atoms
* Equilibrator (Component Contribution) subset that is not in the QM9 data

#### Reaction Prediction data (1200 reactions, 1 property $\Delta G^\circ_{rxn}$)
* Metacyc reactions consisting of only QM9-friendly substrates.

#### Experimental (Golden set) data (~100 reactions)
* NIST free energy of reaction for quantification of true error in prediction


### Accuracy
In our preliminary work, we have been able to extend these findings to enzyme-catalyzed metabolites in the MetaCyc database

![DFT vs MPNN](https://github.com/djinnome/rogain/raw/master/images/DS_cosimulations_JZ/Slide2.png)

Contrast these results with Gibbs free energy of formation using the group contribution method:


![GCM vs MPNN](https://github.com/djinnome/rogain/raw/master/images/DS_cosimulations_JZ/Slide3.png)

As a result, the prediction of Gibbs free energies of formation using MPNN's are as fast as GCM and as accurate as DFT's gaining the best of both worlds.

### Source code

[MPNN code](https://github.com/PNNL-CompBio/graph-neural-network) was reconstructed from text of the [arxiv preprint](http://arxiv.org/abs/1704.01212)

### Proposed work
Rep. of input molecules for prediction of properties: There are a number of things we need to modify in the Message Passing Neural Network (MPNN) in order to predict free energy of reactions involved in metabolism

1. 3D Coordinates: We dont have accurate 3D coordinates of the lowest energy states for MetaCyc/Equilibrator compounds. We know that MPNN gets 11/13 properties within chemical accuracy of DFT using both 3D coordinates of the lowest energy state and the SMILES, whereas it  gets 5/13 properties accurately from the Inchi/Smiles only.

2. Free Energy of formation to free energy of reaction: 
We just have values for the gibbs free energy of formation $\Delta G^\circ_{f}$ and from that we can get free energy of reaction $\Delta G^\circ_{rxn}$ (by calculating $\Delta G_{rxn}^\circ = S^T\Delta G^\circ_{f}$) where $S$ is the stoichiometric matrix of reactions and metabolites.  This will enable us to predict a more accurate free energy of reaction from the QM9 data. The challenge will be how to correctly propagate errors from the Gibbs free energy of formation to the Gibbs free energy of reaction.  We also need to find a data set that contains experimentally measured Gibbs free energies of reactions as a (NIST?)

3. The QM9 dataset contains only molecules with atoms in C, H, N, O, and F.  It hasn't been trained on any molecules with P or S, or with any cofactors. It has also only been trained on molecules with 9 or fewer carbons, so we should not expect to get reasonable results with larger molecules or phosphorylated molecules

We need to extend to multiple NN architectures (try SchNet  and SchNet + edge updates) 



# Predicting Gibbs free energy of the reaction using the catalyst approach

The catalyst approach can be broken down into 3 steps:

* Prediction of enzyme-substrate binding energy
* Prediction of transition energy from enzyme-substrate to enzyme-product
* Prediction of enzyme-product unbinding energy

The sum of these three energies should equal the Gibbs free energy using the solvent approach above.

For the purpose of our project, we focus on the first and third tasks (transition energy from enzyme-substrate to enzyme product can be predicted by subtracting the first and third energies from the energies calculated from the solvent approach)


# Workflow from protein sequence and molecular structure to enzyme-substrate and enzyme-product binding affinities
Our workflow currently includes two independent pathways for generating the enzyme-substrate and enzyme-product binding energies


The first pathway directly predicts binding affinity using protein sequence and chemical structure as input:

* Direct prediction of binding affinity from protein sequence and chemical structure using [PADME](https://arxiv.org/abs/1807.09741)

The second pathway involves predicting the 3D structure, binding the protein to the ligand and estimating the binding affinity:
* Protein sequence to 3D structure using  [Recurrent Geometric Networks](https://www.biorxiv.org/content/early/2018/02/14/265231)
* Ligand binding pocket of 3D structure using  [DeepSite](https://academic.oup.com/bioinformatics/article/33/19/3036/3859178)
* Ligand Pose inside 3D pocket using [Gnina](https://arxiv.org/pdf/1710.07400.pdf)
* Binding affinity of Protein-Ligand complex using [Atomic Convolutional Network](http://arxiv.org/abs/1703.10603)

![Workflow](https://github.com/djinnome/rogain/raw/master/images/DS_cosimulations_JZ.png)


## Direct prediction of binding affinity from protein sequence and chemical structure using PADME

### Neural network architecture
![PADME-NN](https://github.com/djinnome/rogain/raw/master/images/PADME-NN.png)

### Datasets
![PADME-Datasets](https://github.com/djinnome/rogain/raw/master/images/PADME-Datasets.png)
### Accuracy

![PADME-accuracy](https://github.com/djinnome/rogain/raw/master/images/PADME-RMSE-error.png)

### Source code

[PADME](https://github.com/djinnome/PADME) is implemented in Tensorflow using DeepChem, but I have not been able to locate the weights for this model in order to reproduce their results.

### Proposed work
Train PADME (if weights cannot be obtained) using original datasets and use it as a baseline for comparison with the full 3D protein-ligand workflow. Convert the PADME model and weights to ONNX in order to generate an end-to-end model

## Protein sequence to 3D structure using Recurrent Geometric Network

### Neural network architecture
![RGN-NN](https://github.com/djinnome/rogain/raw/master/images/RGN-NN.png)

### Datasets
[ProteinNet](https://github.com/aqlaboratory/proteinnet) is a standardized data set for machine learning of protein structure. It provides protein sequences, structures (secondary and tertiary), multiple sequence alignments (MSAs), position-specific scoring matrices (PSSMs), and standardized training / validation / test splits. ProteinNet builds on the biennial CASP assessments, which carry out blind predictions of recently solved but publicly unavailable protein structures, to provide test sets that push the frontiers of computational methodology. It is organized as a series of data sets, spanning CASP 7 through 12 (covering a ten-year period), to provide a range of data set sizes that enable assessment of new methods in relatively data poor and data rich regimes.
### Accuracy
![RGN-accuracy](https://github.com/djinnome/rogain/raw/master/images/RGN-accuracy.png)

Distributions of mean dRMSD (lower is better, white is median) achieved by servers predicting all structures with >95% coverage at CASP 8-12 are shown for FM (novel folds) and TBM (known folds) categories. Thick black (white on dark background) bars mark RGN dRMSD. CASP 7 is omitted due to lack of server metadata.

![RGN-3d-compare](https://github.com/djinnome/rogain/raw/master/images/RGN-3d-compare.png)

Traces of backbone atoms of well (left), fairly (middle), and poorly (right) predicted RGN structures are shown (bottom) along with their experimental counterparts (top). CASP identifier is displayed above each structure and dRMSD below. A color spectrum spans each protein chain to aid visualization.
### Source Code

The [Recurrent Geometric Network](https://github.com/djinnome/rogain) code was reconstructed  based on the [Biorxiv preprint](https://www.biorxiv.org/content/early/2018/02/14/265231).  The weights are not (yet) available, although the manuscript is in currently in review in Cell Systems.

### Proposed work

Train the RGN using ProteinNet (if weights are not available), and save the output to ONNX so that we can train an end-to-end model

## Ligand binding pocket of 3D structure using DeepSite
### Neural network architecture
![DeepSite-NN](https://github.com/djinnome/rogain/raw/master/images/DeepSite-NN.png)
DeepSite’s internal DCNN architecture. The input is comprised of eight property channels of size 16 Å3. Several convolutional layers with filter size of 8, 4 and 2 reduce the dimensionality of the subgrid before flattening in the 1D dense layers

### Datasets
Datasets used to compare:
* COACH420—consists of 420 single chain structures that contain a mix of drug targets and naturally occurring ligands (we have taken COACH test set [42, 43] and removed proteins contained in CHEN11 and JOINED).

* HOLO4K—large dataset of protein-ligand complexes based on the list published in [79]. Contains larger multi-chain structures downloaded directly from PDB. Disjunct with CHEN11 and JOINED.

### Accuracy
![DeepSiteAccuracy](https://github.com/djinnome/rogain/raw/master/images/DeepSiteAccuracy.png)


![DeepSite-success-rate-v-distance-to-bs](https://github.com/djinnome/rogain/raw/master/images/DeepSite-success-rate-v-distance-to-bs.png)

![DeepSite-pocket-prediction](https://github.com/djinnome/rogain/raw/master/images/DeepSite-pocket-prediction.png)

Fig. 5. Prediction example PDB ID 3KS9. Shaded volume represents DeepSite’s cavity predictions, for which the largest covers most of the annotated ligand


### Source code
[DeepSite source code](https://github.com/djinnome/DeepSite) was obtained from the supplementary material of the [DeepSite paper](https://academic.oup.com/bioinformatics/article/33/19/3036/3859178). It was implemented in PyTorch. Training weights do not appear to be available, but a web-based version is available for free to validate our predictions

### Proposed work
If the PlayMolecule folks will not provide the weights, we will pre-train DeepSite on the COACH420 and HOLO4K data set and validate with the PlayMolecule.org instance results. We will also likely need to convert the PyTorch instance model and weights to ONNX so that we can train an end-to-end architecture 









## Ligand Pose inside 3D pocket using Gnina

### Neural network architecture
![Gnina-NN](https://github.com/djinnome/rogain/raw/master/images/Gnina-NN.png)
### Dataset

All datasets were derived from the 2016 version of the PDBbind general set [48], which contains 13,308 complexes of ligands bound to target proteins from the Protein Data Bank [2] with structures determined by X-ray crystallography. Certain complexes were deemed unsuitable due to having unusually high energy or being unusually large. The internal energy of each ligand in the general set was calculated with Open Babel using the default MMFF94 force field. Those with energy greater than 10,000 kJ/mol were removed from the data set. The molecular weight was also calculated using Open Babel and ligands with weight greater than 1200 Da were removed. This resulted in a modified general set containing 12,482 structures. We then generated an initial training set by redocking the crystal structures with AutoDock Vina and taking approximately 20 of the top-ranked poses per target. We additionally included poses that resulted from local optimization of the crystal structure poses with AutoDock Vina, to ensure that each target had at least one correct binding pose in the training set while avoiding training directly on the crystal structures. Poses were labeled based on their root mean squared distance (RMSD) from the crystal pose. Any pose less than 2Å RMSD from the crystal pose was labeled as a binding pose, and any pose greater than 4Å RMSD was labeled as a non-binding pose. Poses between 2 and 4Å RMSD were considered ambiguous, and were not included in the training set. The distribution of poses in the training set can be seen in Figure 1 and Table 1.
3.2
![Gnina-Histogram](https://github.com/djinnome/rogain/raw/master/images/Gnina-histogram-of-RMSD.png)
![Gnina-Dataset](https://github.com/djinnome/rogain/raw/master/images/Gnina-Dataset.png)
### Accuracy
![Gnina-pose](https://github.com/djinnome/rogain/raw/master/images/Gnina-pose-compare.png)
![Gnina-accuracy](https://github.com/djinnome/rogain/raw/master/images/Gnina-accuracy.png)
## Binding affinity of Protein-Ligand complex using Atomic Convolutional Network
### Neural net architecture
![AtomicConvNet-NN](https://github.com/djinnome/rogain/raw/master/images/AtomicConvNet-NN.png)

![AtomicConvNet-DeltaG](https://github.com/djinnome/rogain/raw/master/images/AtomicConvNet-DeltaG.png)

### Dataset
PDBBind is a database of experimentally measured binding affinities for protein ligand complexes. PDBBind provides 3D crystal structures and associated inhibition constant Ki for the protein-ligand complexes in its collection. The PDBBind 2015 dataset contains three subsets: core (195 structures), refined (3,706 structures), and full (14,260). The crystal structures present in the refined datasets are obtained at a higher resolution and cleaned more thoroughly than the full dataset, in addition to more stringent requirements on the quality of the complex structure, quality of the binding data, and the nature of the complex. The complexes present in the refined set are then clustered by 90% similarity in protein sequence to create 65 families. From these families, the core set was created by selecting three representative complexes (weakest binding, median binding, and strongest binding) to control sample redundancy. Additional information on the curation of these datasets can be found elsewhere25. We use the core and refined subsets to train and benchmark the performance of ACNNs. Other popular protein-ligand scoring functions trained on the PDBBind dataset include X-Score26 and AutoDock Vina4.
Dataset
### Accuracy

In all ACNN core set models, the mean absolute error on the test set is less than 1 kcal/mol, a barrier which has been
previously cited as chemical accuracy necessary to enable rapid computational drug design.[Peterson 2012](https://link.springer.com/article/10.1007/s00214-011-1079-5)
![AtomicConvNet-accuracy](https://github.com/djinnome/rogain/raw/master/images/AtomicConvNet-accuracy.png)

### Source code 

[Gnina](https://github.com/djinnome/gnina) is implemented in a forked version of Caffe (not Caffe2). 

### Proposed work
Fortunately, there are methods for reading in the prototext model and weights into PyTorch. Unfortunately, new layer types have been created for Gnina (specifically the `MolGridData` layer type) which may complicate that import. We may need additional software engineering expertise to export Gnina into ONNX.





# Rogain End-to-end architecture

Once all models have been exported to ONNX, the next step is to train the end-to-end architecture 

## Neural net architecture



![Workflow](https://github.com/djinnome/rogain/raw/master/images/DS_cosimulations_JZ.png)


## Dataset

### Drug Target Commons
![DTC](https://github.com/djinnome/rogain/raw/master/images/E2E_DTC_Dataset.png)
### PDBBind


### BindDB

### MOAB

## Accuracy

We will compare the predictions we get using PADME against the predictions we get using our Rogain End-to-end workflow.
We will also compare the results that others get from the DREAM competition, which is using the same training set.
Lastly, we will compare the predictions we get for the Enzyme-substrate and enzyme product transitions against a dataset of actual enzyme:substrate to enzyme:product energies.

## Source code

Since some of the Neural networks that comprise `rogain` were written in pytorch, others in Tensorflow, and still others in a forked version of Caffe, we will first export all neural networks into an intermediate representation called ONNX.

## Proposed work, 
If any of the exports fail, then we will import all the other models into that language.  At this point, I am guessing that `GNINA` will be the bottleneck. which means converting everything to `Caffe`.  If not, then Tensorflow is the preferred language.





